In [ ]:
from extraction import extract_articles_from_google
from extraction import extract_data_from_page
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By

In [ ]:
pages_file_name = "la_cuarta_articles.json"
content_file_name = "la_cuarta_articles_with_content.json"

In [ ]:
extract_articles_from_google(
	website="lacuarta.com",
	newscast="La Cuarta",
    pages=2,
    output=pages_file_name,
    keywords="Estallido Social",
    )

In [ ]:
def get_content(body: WebElement):
	content = ""
	for element in body.find_elements(By.CLASS_NAME, "article-body__paragraph"):
		content += element.text.strip() + "\n"
	return content

def get_author(body: WebElement):
	try:
		return body.find_element(By.CLASS_NAME, "article-body__byline__author").text.strip()
	except:
		return "not found"

def get_description(body: WebElement):
	try:
		return body.find_element(By.CLASS_NAME, "article-head__subtitle").text.strip()
	except:
		return "not found"


extract_data_from_page(
	input_file=pages_file_name, 
	output_file=content_file_name,
	get_author=get_author,
	get_description=get_description,
	get_content=get_content,
	limit_of_pages=5
)

In [ ]:
json_pages_info = {"pages": []}

In [ ]:
import pyautogui
import random

months = {
    "ene": "Jan", "feb": "Feb", "mar": "Mar", "abr": "Apr",
    "may": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug",
    "sept": "Sep", "oct": "Oct", "nov": "Nov", "dic": "Dec"
}

locale.setlocale(locale.LC_TIME, 'C')

# json_pages_info = {"pages": []}

pages = 20

re = False

for page in range(10, pages):

	first = True
	while (re or first):
		first = False
		
		try: 
			time.sleep(1)

			if (not re): 
				options = uc.ChromeOptions()
				options.add_argument("--no-sandbox")
				options.add_argument("--disable-blink-features=AutomationControlled")

				driver = uc.Chrome(options=options)

				url = f'https://www.google.com/search?q=%22Estallido+social%22+site%3Awww.lacuarta.com&tbs=cdr:1,cd_min:11/15/2019,cd_max:12/17/2023&start={page * 10}'
				driver.get(url)

			re = False
			time.sleep(random.uniform(6, 12))

			articles_section = driver.find_element(By.CLASS_NAME, "dURPMd")
			articles = articles_section.find_elements(By.CLASS_NAME, "MjjYud")

			for article in articles:
				print("revisando artículo...")
				try: 
					originalDate = article.find_element(By.CLASS_NAME, "YrbPuc").find_element(By.TAG_NAME, "span").text
					for es, en in months.items():
						if es in originalDate:
								originalDate = originalDate.replace(es, en)
								break
						
					date_epoch = int(time.mktime(time.strptime(originalDate, "%d %b %Y")))

					if (date_epoch < 1573786800 or date_epoch > 1702782000):
						print("La noticia no corresponde a la fecha solicitada")
						continue

					# check if the article is from El Mostrador
					if (not ("www.lacuarta.com" in article.text)):
						print("El artículo no es de La Cuarta")
						continue

					try:
						title = article.find_element(By.TAG_NAME, "h3").text
						description = article.find_element(By.CLASS_NAME, "kb0PBd ").find_elements(By.TAG_NAME, "span")[1].text
						link = article.find_element(By.TAG_NAME, "a").get_attribute("href")

						link_info = {
							"newscast" : "La Cuarta",
							"title": title,
							"description": description,
							"category": "The site does not provide a category",
							"date": originalDate,
							"image_link": "not found initially",
							"author": "not found initially",
							"link": link,
						}

						print("información incluida!")
						json_pages_info["pages"].append(link_info)

					except Exception as e:
						print(f"Error al extraer información del artículo: {e}")
						continue

				except Exception as e:
					print(f"Error al procesar el artículo: {e}")
					continue
		except Exception as e:
			print(f"Error al procesar el resultado de busqueda. Página: {page}")
			print(f"Sitio: {url}")
			print(e)
			response = input("Reintentar? y/n")
			if response.lower() == 'y':
				print("Reintentando.")
				re = True
			else: 
				print("Cancelando...")
				re = False
		
		if (not re):
			driver.quit()

print("Almacenando información en el archivo JSON...")
with open(f"../archive/temp/la_cuarta.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


# with open(f"output.html", "w", encoding="utf-8") as f:
# 	f.write(driver.page_source)

# Content Extraction

In [ ]:
import pyautogui
import random

months = {
    "ene": "Jan", "feb": "Feb", "mar": "Mar", "abr": "Apr",
    "may": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug",
    "sept": "Sep", "oct": "Oct", "nov": "Nov", "dic": "Dec"
}

locale.setlocale(locale.LC_TIME, 'C')

json_pages_info = {"pages": []}

pages = 1

re = False

for page in range(0, pages):

	first = True
	while (re or first):
		first = False
		
		print("procesando")

		try: 
			time.sleep(1)

			if (not re): 
				options = uc.ChromeOptions()
				options.add_argument("--no-sandbox")
				options.add_argument("--disable-blink-features=AutomationControlled")

				driver = uc.Chrome(options=options)

				url = f'https://www.lacuarta.com/cronica/noticia/los-50-dias-del-estallido-social/435927/'
				print("obteniendo URL")

				driver.get(url)

			re = False
			print("Time sleep de 6 a 12")
			time.sleep(random.uniform(6, 12))

			title = driver.find_element(By.TAG_NAME, "h1").text
			print(f"Procesando página: {page + 1} - Título: {title}")

		except Exception as e:
			print(f"Error al procesar el resultado de busqueda. Página: {page}")
			print(f"Sitio: {url}")
			print(e)
			response = input("Reintentar? y/n")
			if response.lower() == 'y':
				print("Reintentando.")
				re = True
			else: 
				print("Cancelando...")
				re = False
		
		if (not re):
			driver.quit()

#print("Almacenando información en el archivo JSON...")
#with open(f"../archive/temp/la_cuarta.json", 'w', encoding='utf-8') as file:
#	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


In [ ]:

extracted_pages = {"pages": []}
extracted_pages_with_content = {"pages": []}
last_extracted_pages_with_content = {"pages": []}

total_pages = 0
pages_succeeded = 0
pages_failed = 0
error_occurred = False
driver = None
limit_of_pages = 0
start_page = 0

content = ""
author = ""
description = ""
raw_json_content = {}

auto_response_count = 0

test_specific_url = ""

def get_response():
	return input("Retry? y/n: ")

print("[INFO] Loading extracted pages from JSON file...")
with open("../archive/pages_extracted/la_cuarta/la_cuarta_240725_cleanData.json", 'r', encoding='utf-8') as file:
	extracted_pages = json.load(file)
	print(f"[INFO] {len(extracted_pages['pages'])} pages loaded.")
	if limit_of_pages == "all":
		limit_of_pages = len(extracted_pages["pages"])
		print(f"[INFO] Limit of pages set to {limit_of_pages}")

def recursive_search(element):
	element_tag_name = element.tag_name
	print(element_tag_name)
	print(element.text)
	if element_tag_name == "p" or element_tag_name == "span" or element_tag_name == "h2" or element_tag_name == "h3" or element_tag_name == "h4" or element_tag_name == "a":
		element.text
		return element.text.strip()
	
	children = element.find_elements(By.XPATH, ".//*")
	for child in children:
		result = recursive_search(child)
		if result:
			return result
	return ""


# with open("../archive/temp/pages_with_content/la_tercera_with_content_reference.json", 'r', encoding='utf-8') as file:
# 	last_extracted_pages_with_content = json.load(file)
# 	print(f"[INFO] {len(last_extracted_pages_with_content['pages'])} pages loaded.")


if start_page >= len(extracted_pages["pages"]):
	print(f"[ERROR] start_page is out of range. The maximum page index is {len(extracted_pages['pages']) - 1}.")
	raise SystemExit

def extract_from_script():
		print("[INFO] Trying to extract content from <script> tags...")
		data = {"content": "", "description": "", "raw_json_content": ""}

		scripts = driver.find_elements(By.XPATH, "//head/script")
		data_in_script = {}
		content_extracted_by_json = False
		for script in scripts: 
			script_content = script.get_attribute("innerHTML").strip()

			# Intentar interpretarlo como JSON
			is_json = False
			content_extracted_by_json = False
			try:
					data_in_script = json.loads(script_content)   # si es JSON válido, lo convierte en dict/list
					test = data_in_script["articleBody"]
					if test is not None:
						print("[INFO] Content found in <script> tag.")
					is_json = True
			except json.JSONDecodeError:
					is_json = False

			except Exception as e:
					is_json = False
			
			try:
				if is_json:
						data["content"] = data_in_script["articleBody"]
						data["raw_json_content"] = data_in_script
						
						try:
							data["description"] = data_in_script["description"]
						except Exception as e:
							print("[ERROR] A error occurred while extracting description", flush=True)
							data["description"] = "not found"


						content_extracted_by_json = True
						print("[INFO] The content has been extracted. Method: JSON", flush=True)
						break
			except Exception as e:
					print(f"[ERROR] A error occurred while extracting content from JSON script: {e}", flush=True)
					content_extracted_by_json = False

		return content_extracted_by_json, data

def extract_from_DOM():
	print("[INFO] Trying to extract content from DOM...")
	content = ""
	author = ""
	description = ""

	# paragraph = content_container.find_elements(By.TAG_NAME, "p") if content_container else None
	paragraph_elements = body.find_elements(By.CLASS_NAME, "article-body__paragraph")
	content = ""

	for element in paragraph_elements:
		p = recursive_search(element)
		if "Volver al Home" in p or "También puedes leer" in p:
			continue
		content += p + "\n"

	try:
		description = body.find_element(By.CLASS_NAME, "article-head__subtitle").text.strip()
	except:
		description = "not found"

	print("[INFO] The content has been extracted. Method: DOM", flush=True)
	if len(content) < 100:
			print("[WARNING] The extracted content is too short.", flush=True)
			print(f"[WARNING] Content: {content}", flush=True)
	return content, author, description

data_extracted = True
re = False
auto_re = False

def get_driver():
	options = uc.ChromeOptions()
	options.add_argument("--no-sandbox")
	options.add_argument("--disable-blink-features=AutomationControlled")

	driver = uc.Chrome(options=options)
	driver.set_page_load_timeout(15)
	return driver

real_page_index = 0

print("[INFO] Starting extraction process...")
for page_index, page in enumerate(extracted_pages["pages"]):

		if test_specific_url == "":
			if total_pages >= limit_of_pages:
					print("[INFO] Limit of pages reached.")
					break

			if "content" not in page.keys():
					page["content"] = "The content has not been extracted yet"

			if page_index < start_page:
					# if page_index < len(last_extracted_pages_with_content["pages"]):
					# 		extracted_pages_with_content["pages"].append(last_extracted_pages_with_content["pages"][page_index])
					# else:
					# 	extracted_pages_with_content["pages"].append(page)
					real_page_index = real_page_index + 1
					print("[INFO] Skipping page: ", page_index)
					continue
			if page["content"] != "The content has not been extracted yet" and page["content"] != "": 
					print(f"[INFO] Page {page_index} already has content. Skipping...")
					extracted_pages_with_content["pages"].append(page)
					real_page_index = real_page_index + 1
					continue
		else:
			print("[INFO] Processing specific page: ", test_specific_url)
			for page_aux in extracted_pages["pages"]:
				if page_aux["link"] == test_specific_url:
					page = page_aux
					test_specific_url = ""
					total_pages = 99999
					print("[INFO] The specific page found!")
					break

		total_pages += 1

		print(f"\n[INFO] Checking page: {page_index} | Link: {page['link']}" )
		url = page['link']
		first = True
		auto_re = False
		auto_response_count = 0

		while (re or first):
			first = False
			
			try: 
				time.sleep(1)

				if (not re): 
	
					driver = get_driver()
					print("[INFO] Chrome driver initialized.")
					driver.get(url)
					print("[INFO] Page loaded.")

				re = False
				time.sleep(random.uniform(6, 12))
				body = driver.find_element(By.TAG_NAME, 'body')

				try: 
					content_extracted_by_json, data = extract_from_script()
					if not content_extracted_by_json: 
						content, author, description = extract_from_DOM()

						if author == "not found":
							try:
								author = driver.find_element(By.CLASS_NAME, "a_md_a").text.strip()
							except:
								author = "not found"
					else:
						content = data["content"]
						description = data["description"]
						raw_json_content = data["raw_json_content"] if data["raw_json_content"] != "" else "not found initially"

					try:
						author_meta = driver.find_element(By.CSS_SELECTOR, "meta[property='article:author']")
						author = author_meta.get_attribute("content")
					except Exception as e:
						print("[ERROR] A error occurred while extracting author name", flush=True)
						print("[INFO] Setting author to 'not found'", flush=True)
						print(e, flush=True)
						author = "not found"

				except Exception as e:
					print(f"[ERROR] A error occurred while extracting content", flush=True)
					raise

			except Exception as e:
				print(f"[ERROR] A error occurred while processing the page: {page_index}", flush=True)
				print(f"[ERROR] Site: {url}", flush=True)
				print(e, flush=True)

				# response = input("Retry? y/n: ")
				time.sleep(10)
				try:
					if auto_response_count == 3:
						response = "n"
					else:
						
						driver.refresh()
						time.sleep(5)
						response = "y"
					auto_response_count += 1

					if response.lower() == 'y':
						print("[INFO] Retry...")
						re = True
					elif response.lower() == "n" or response == "":  
						# response = input("Description? Press enter to not add a description")
						response = "[AUTO] Limit of attempts reached"
						if not response:
								response = "No description provided"
						else: 
								print(f"[INFO] The user add a description: {response}")
						print("[INFO] Cancelling...")
						error_occurred = True
						page["Observation"] = "A error occurred while extracting content. The user cancelled the operation. Description: " + response
						re = False
					else:
						print("[INFO] The response is unknown.")
						print("[INFO] Cancelling...")
						error_occurred = True
						re = False
				except Exception as e:
					print("[ERROR] a error occurred while processing the page:", flush=True)
					print(f"[ERROR] Site: {url}", flush=True)
					print(e, flush=True)
					re = False
					error_occurred = True
					page["Observation"] = "A error occurred while extracting content. Can't retrieve the content."

		if (not re):
			driver.quit()

		if error_occurred:
			content = "A error occurred while extracting content"
			author = "A error occurred while extracting content"
			description = "A error occurred while extracting content"

			pages_failed += 1
			error_occurred = False
		else:
			pages_succeeded += 1

		page["content"] = content
		page["author"] = author
		page["description"] = description
		page["raw_json_content"] = raw_json_content
		extracted_pages_with_content["pages"].append(page)

		content = ""
		author = ""
		description = ""
		raw_json_content = {}

		print("[INFO] Storing information in a JSON file...")
		with open(f"../archive/temp/pages_with_content/la_cuarta/la_cuarta_with_content-{real_page_index}.json", 'w', encoding='utf-8') as file:
			json.dump(extracted_pages_with_content, file, ensure_ascii=False, indent=4)

		real_page_index = real_page_index + 1

		extracted_pages_with_content = {"pages": []}

		driver.quit()

		time.sleep(random.uniform(6, 12))

print("Finished processing pages.")

# ----------------- Stats -----------------
print("\n\n")
print("----- STATS -----")

try:
	print(f"Amount of pages processed: {total_pages}")
	print(f"% Pages succeeded [{pages_succeeded}]: {round(100 * pages_succeeded/total_pages, 2)}")
	print(f"% Pages failed [{pages_failed}]: {round(100 * pages_failed/total_pages, 2)}")
except: 
	print("A error occurred while processing the stats")